In this notebook we'll explore feature importance using SHAP values. SHAP values are the most mathematically consistent way for getting feature importances, and they work particulalry nicely with the tree-based models. Unfortunately, calculating SHAP values is an **extremely** resource intensive process. However, starting with XGBoost 1.3 it is possible to calcualte these values on GPUs, whcih speeds up the process by a factor of 20X - 50X compared to calculating the same on a CPU. Furthermore, it is also possible to calculate SHAP values for feature interactions. The GPU speedup for those is even more dramatic - it takes a few minutes, as opposed to days or even longer on a CPU.

In [ ]:
%matplotlib inline


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
import shap
import xgboost as xgb

# load JS visualization code to notebook
shap.initjs()

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')


In [ ]:
columns = test.columns[1:]
columns


In [ ]:
target = train['target'].values

Let's applay label encoder to the categorical features.

In [ ]:
cat_features = columns[:10]
cat_features

In [ ]:
for feature in cat_features:
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train_oof = np.zeros((train.shape[0],))
test_preds = 0
train_oof_shap = np.zeros((train.shape[0],train.shape[1]-1))
test_preds_shap = 0
train_oof.shape

In [ ]:
xgb_params= {
        "objective": "reg:squarederror",
        "max_depth": 6,
        "learning_rate": 0.01,
        "colsample_bytree": 0.4,
        "subsample": 0.6,
        "reg_alpha" : 6,
        "min_child_weight": 100,
        "n_jobs": 2,
        "seed": 2001,
        'tree_method': "gpu_hist",
        "gpu_id": 0,
        'predictor': 'gpu_predictor'
    }

In [ ]:
test = xgb.DMatrix(test[columns])

In [ ]:
NUM_FOLDS = 10
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        train_target, val_target = target[train_ind], target[val_ind]
        
        train_df = xgb.DMatrix(train_df, label=train_target)
        val_df = xgb.DMatrix(val_df, label=val_target)
        
        model =  xgb.train(xgb_params, train_df, 3800)
        temp_oof = model.predict(val_df)
        temp_oof_shap = model.predict(val_df, pred_contribs=True)
        temp_test = model.predict(test)
        temp_test_shap = model.predict(test, pred_contribs=True)

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        train_oof_shap[val_ind, :] = temp_oof_shap
        test_preds_shap += temp_test_shap/NUM_FOLDS
        
        print(mean_squared_error(temp_oof, val_target, squared=False))

In [ ]:
0.8434888431362352

In [ ]:
mean_squared_error(train_oof, target, squared=False)


In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)
np.save('train_oof_shap', train_oof_shap)
np.save('.test_preds_shap', test_preds_shap)

Next, we calculate the SHAP values for the test set.



Now let's do some plots of these values.

Next, we'll calculate SHAP values for featue interactions. There will be 24x24x200,000 + 200,000 numbers that need to be computed.

It took less than 2 minutes to calculate these values. On CPU this would take up to a day to compute.

Now let's take a look at what are the top interactions in this dataset.

We will now construct 5 more features out of the top 5 most interactive ones.

Now let's see what are the SHAP values for this model